***OBJECTIVES 1***

- using training set, calculate recovery for (rougher.output.recovery), find MAE between calculations and actuals
- which features are not availavle in the test set? what are they? their type?
- perfrom preprocessing, duplicates? missing?, feature scaling

***NOTES***

1. recovery(ROUGHER.output.recovery) = ((rougher.output.concentrate_au * (rougher.input.feed_au - rougher.output.tail_au)) / (rougher.input.feed_au * (rougher.output.concentrate_au - rougher.output.tail_au))) * 100%

2. recovery(FINAL.output.recovery) = ((final.output.concentrate_au * (rougher.output.concentrate_au - secondary_cleaner.output.tail_au)) / (rougher.output.concentrate_au * (final.output.concentrate_au - secondary_cleaner.output.tail_au))) * 100%

**elements**
- Au = gold
- Ag = silver
- Pb = lead

In [24]:
import pandas as pd
import numpy as np

data_full = pd.read_csv('gold_recovery_full.csv')
data_train = pd.read_csv('gold_recovery_train.csv')
data_test = pd.read_csv('gold_recovery_test.csv')

# list of features used for rougher recovery calculation step
rougher_recovery_features = ['rougher.output.concentrate_au', 'rougher.input.feed_au', 'rougher.output.tail_au', 'rougher.output.recovery']

# displaying non-missing/missing values and their combinations, including percentage as decimal
print(data_train[rougher_recovery_features].isna().value_counts())
print(data_train[rougher_recovery_features].isna().sum())
print(data_train[rougher_recovery_features].isna().sum() / len(data_train))

#print(data_full.info())
#print(data_train.info())
#print(data_test.info())
#print(data_full.isna().sum())
#print(data_train.isna().sum())
#print(data_test.isna().sum())
#print(data_full.sample(20, random_state=777))

rougher.output.concentrate_au  rougher.input.feed_au  rougher.output.tail_au  rougher.output.recovery
False                          False                  False                   False                      14287
                                                      True                    True                        2167
                                                      False                   True                         323
True                           True                   True                    True                          82
False                          True                   False                   True                           1
Name: count, dtype: int64
rougher.output.concentrate_au      82
rougher.input.feed_au              83
rougher.output.tail_au           2249
rougher.output.recovery          2573
dtype: int64
rougher.output.concentrate_au    0.004864
rougher.input.feed_au            0.004923
rougher.output.tail_au           0.133393
rougher.output.recovery   

**- All 3 data sets are missing values "randomly" in almost every column, with ['date'] being the functional index seeing as though none of the datetime columns are missing any values in any of the data sets**

**- There are 2,573 rows we cannot use to validate rougher.output.recovery due to at least one parameter missing, I will be creating a separate df containing only non-missing recovery parameters**

In [33]:
# df containing non-null parameters and output recovery
rougher_recovery_df = data_train[rougher_recovery_features].dropna()

# simplified variable capture for calculation
C = rougher_recovery_df['rougher.output.concentrate_au']
F = rougher_recovery_df['rougher.input.feed_au']
T = rougher_recovery_df['rougher.output.tail_au']

# where recovery denominator != 0, calculate recovery, otherwise 0 ... create new column with vectorization
calculated_recovery = np.where(F * (C - T) != 0, (C * (F - T)) / (F * (C - T)) * 100, 0)
rougher_recovery_df['calculated_rougher_recovery'] = calculated_recovery

# import MAE and verify accuracy of data
from sklearn.metrics import mean_absolute_error as mae

actual = rougher_recovery_df['rougher.output.recovery']
calculated = rougher_recovery_df['calculated_rougher_recovery']

print('Mean Absolute Error: ', mae(actual, calculated))

Mean Absolute Error:  9.303415616264301e-15


**- With an MAE so close to zero, we can see that the existing data correctly implements the recovery equation to find output values**